In [4]:
import numpy as np
import pywt
import librosa, soundfile
from tqdm import tqdm

# free sound effects
# https://pixabay.com/

In [11]:
inputFile = 'freenoisesample.mp3'
outputFile = 'filtered_wavelet.wav'
y, rate = librosa.load(inputFile, sr=None, mono=False)
duration = librosa.get_duration(y=y, sr=rate)

# copied from 
# https://github.com/ap-atul/Audio-Denoising
def mad(arr):
    """ Median Absolute Deviation: a "Robust" version of standard deviation.
        Indices variability of the sample.
        https://en.wikipedia.org/wiki/Median_absolute_deviation 
    """
    arr = np.ma.array(arr).compressed()
    med = np.median(arr)
    return np.median(np.abs(arr - med))
with soundfile.SoundFile(outputFile, "w", samplerate=rate, channels=y.shape[0]) as of:
    for block in tqdm(soundfile.blocks(inputFile, int(rate * duration * 0.10))):
                coefficients = pywt.wavedec(block, 'db4', mode='per', level=2)

                #  getting variance of the input signal
                sigma = mad(coefficients[- 1])

                # VISU Shrink thresholding by applying the universal threshold proposed by Donoho and Johnstone
                thresh = sigma * np.sqrt(2 * np.log(len(block)))

                # thresholding using the noise threshold generated
                coefficients[1:] = (pywt.threshold(i, value=thresh, mode='soft') for i in coefficients[1:])

                # getting the clean signal as in original form and writing to the file
                clean = pywt.waverec(coefficients, 'db4', mode='per')
                of.write(clean)


print("hello")

11it [00:06,  1.66it/s]

hello
